In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi'] = RSI(data['close'])

from talib import MA, SMA, EMA
data['ma'] = MA(data['close'])

In [2]:
data['close+'] = data['close'].shift(-1)
data.dropna(axis=0, inplace=True)

In [3]:
data

,timestamp,date,open,high,low,close,rsi,ma,close+
29,1612879200,2021-02-09 14:00:00,1743.07,1760.89,1735.27,1760.23,60.726497,1710.117667,1751.09
30,1612882800,2021-02-09 15:00:00,1760.23,1761.00,1743.88,1751.09,58.774198,1715.234333,1736.78
31,1612886400,2021-02-09 16:00:00,1751.10,1765.00,1721.93,1736.78,55.752107,1720.006667,1744.33
32,1612890000,2021-02-09 17:00:00,1736.66,1751.13,1734.10,1744.33,57.008134,1725.372667,1754.75
33,1612893600,2021-02-09 18:00:00,1744.34,1761.43,1739.39,1754.75,58.748543,1729.864333,1770.48
...,...,...,...,...,...,...,...,...,...
16256,1675785600,2023-02-07 16:00:00,1630.44,1644.32,1623.72,1637.98,51.458206,1633.656000,1638.52
16257,1675792800,2023-02-07 18:00:00,1672.03,1679.03,1625.48,1638.52,51.870094,1633.974000,1656.42
16258,1675796400,2023-02-07 19:00:00,1638.52,1658.48,1637.64,1656.42,63.059562,1634.561333,1664.93
16259,1675800000,2023-02-07 20:00:00,1656.43,1668.77,1650.16,1664.93,66.988870,1635.701000,1667.43


In [4]:
inputs = data.iloc[:, [5,7]]
target = data.iloc[:, [8]]

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2, random_state=1, shuffle=False)

In [6]:
X_train.shape[0]

12985

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load and preprocess the data
# Assuming 'data' is a DataFrame containing historical prices and moving averages
# Preprocess the data, scale the features, and split into training and testing sets

# Define the LSTM model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss:", loss)

# Make predictions
predictions = model.predict(X_test)

2024-02-26 06:35:23.498720: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 06:35:24.634138: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 06:35:24.634439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 06:35:24.840063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 06:35:25.270495: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 06:35:29.686694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

IndexError: tuple index out of range